In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

consumer_history = pd.read_csv('/kaggle/input/zb-da9-competition/ConsumerElectronics.csv')
ad_investment = pd.read_csv('/kaggle/input/zb-da9-competition/Media Investment.csv')
nps_stockindex = pd.read_csv('/kaggle/input/zb-da9-competition/NPS_Stockindex.csv')
promotion_data = pd.read_csv('/kaggle/input/zb-da9-competition/Special_Sale_Calendar.csv')

In [ ]:
consumer_history.head()

In [ ]:
consumer_history.info()

In [ ]:
consumer_history.rename(columns= {'s1_fact.order_payment_type' : 'order_payment_type'}, inplace=True)
consumer_history['order_date'] = pd.to_datetime(consumer_history['order_date'])
consumer_history.info()

In [ ]:
# 첫 구매 고객 계산
consumer_history['First_purchase'] = consumer_history.groupby(['cust_id'])['order_date'].transform('min')

# Cohort index 생성
consumer_history['Year'] = consumer_history['order_date'].dt.year
consumer_history['Month'] = consumer_history['order_date'].dt.month
consumer_history['First_Year'] = consumer_history['First_purchase'].dt.year
consumer_history['First_Month'] = consumer_history['First_purchase'].dt.month
consumer_history['cohortindex'] = (consumer_history['Year'] - consumer_history['First_Year']) * 12 + (consumer_history['Month'] - consumer_history['First_Month']) + 1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cohort_data = consumer_history.groupby(['First_Year', 'First_Month', 'cohortindex']).agg({'cust_id': 'nunique'}).reset_index()
cohort_pivot = cohort_data.pivot_table(index=['First_Year', 'First_Month'], columns='cohortindex', values='cust_id', fill_value=0)

# 코호트 시각화
plt.figure(figsize=(12, 8))
sns.heatmap(cohort_pivot, annot=True, fmt='g', cmap='YlGnBu', cbar_kws={'label': 'Number of New Customers'})
plt.title('Customer Cohort Analysis Heatmap')
plt.xlabel('Months Since First Purchase') # 첫 구매 이후 지난 개월
plt.ylabel('Year-Month of First Purchase') # 첫 구매 년도 - 월
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 분석 일자 추출 (2015년 09월 ~ 2016년 03월)
cohort_pivot = cohort_pivot.loc[(cohort_pivot.index.get_level_values('First_Year') == 2015) & 
                                (cohort_pivot.index.get_level_values('First_Month') >= 9) |
                                (cohort_pivot.index.get_level_values('First_Year') == 2016) & 
                                (cohort_pivot.index.get_level_values('First_Month') <= 3)]
# 첫 구매자 수 (0개월 차) 가져오기
cohort_sizes = cohort_pivot.iloc[:, 0]

In [ ]:
# 재구매율 계산
retention_rate = cohort_pivot.divide(cohort_sizes, axis=0) * 100

# 리텐션율 히트맵 시각화
plt.figure(figsize=(12, 8))
sns.heatmap(retention_rate, annot=True, fmt=".1f", cmap="YlGnBu", cbar_kws={'label': 'Retention Rate (%)'})
plt.title('Customer Cohort Retention Rate Heatmap')
plt.xlabel('Cohort Index (Months Since First Purchase)')
plt.ylabel('Cohort (Year-Month of First Purchase)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 이탈률 계산: 100%에서 재구매율을 뺀 값
purchase_Out_rate = 100 - retention_rate

# 이탈율 히트맵 시각화
plt.figure(figsize=(12, 8))
sns.heatmap(purchase_Out_rate, annot=True, fmt=".1f", cmap="Reds", cbar_kws={'label': 'Purchase Out Rate (%)'})
plt.title('Customer Purchase Out Rate Heatmap')
plt.xlabel('Cohort Index (Months Since Repurchase)')
plt.ylabel('Cohort (Year-Month of Repurchase)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 그룹 1: 첫 구매일이 2015-09 ~ 2015-11인 고객
group1 = consumer_history[(consumer_history['First_purchase'] >= '2015-09-01') &
                          (consumer_history['First_purchase'] <= '2015-11-30')]


# 그룹 2: 첫 구매일이 2015-12 ~ 2016-03인 고객
group2 = consumer_history[(consumer_history['First_purchase'] >= '2015-12-01') &
                          (consumer_history['First_purchase'] <= '2016-02-28')]

# 중복값 제거 후, 고객 ID만 추출
group1 = group1[['cust_id']].drop_duplicates()
group2 = group2[['cust_id']].drop_duplicates()

# 고객 ID 고유 개수 확인
display(group1.shape)  # 그룹 1의 고유 고객 수
display(group2.shape)  # 그룹 2의 고유 고객 수

### Group 1(첫 구매일이 2015-09 ~ 2015-11인 고객): 363,896명 - 첫 구매 이후 재구매가 현저히 낮은 그룹

### Group 2(첫 구매일이 2015-12 ~ 2016-03인 고객): 365,447명 - 첫 구매후 재구매 행동을 어느정도 보이는 그룹

In [ ]:
consumer_history['order_date'].min() # 2015-05-19 13:42:09
consumer_history['order_date'].max() # 2016-07-25 01:19:45

# 2015-09-01 ~ 2016-03-31까지 데이터 필터링
consumer_history = consumer_history[consumer_history['order_date'] >= '2015-09-01']
consumer_history = consumer_history[consumer_history['order_date'] <= '2016-02-28']

# 필터링한 데이터에서 2016-03-30 23:59:25를 기준으로 설정
reference_date = consumer_history['order_date'].max()
print("기준 날짜 (Reference Date):", reference_date)

# 고객별 최근 거래 일자를 계산 (마지막 거래일)
consumer_history['last_order'] = consumer_history.groupby('cust_id')['order_date'].transform('max')

print(consumer_history['order_date'].min())
print(consumer_history['order_date'].max()) 


# RFM 분석

---

In [ ]:
# 그룹 1: 첫 구매일이 2015-09-01 ~ 2015-11-30
group1 = consumer_history[(consumer_history['First_purchase'] >= '2015-09-01') &
                        (consumer_history['First_purchase'] <= '2015-11-30')]['cust_id'].unique()
group1_data = consumer_history[consumer_history['cust_id'].isin(group1)]

# 그룹 2: 첫 구매일이 2015-12-01 ~ 2016-02-28인 고객의 ID 목록
group2 = consumer_history[(consumer_history['First_purchase'] >= '2015-12-01') &
                        (consumer_history['First_purchase'] <= '2016-02-28')]['cust_id'].unique()
group2_data = consumer_history[consumer_history['cust_id'].isin(group2)]

In [ ]:
display(group1_data.columns)
display(group2_data.columns)

In [ ]:
consumer_history['gmv'] = consumer_history['gmv'].fillna(0)
consumer_history['gmv'] = pd.to_numeric(consumer_history['gmv'], errors='coerce').fillna(0)
consumer_history['gmv'] = consumer_history['gmv'].astype(int)

#gmv (Monetary) 값 확인: NaN, 0 또는 음수 값 제거
consumer_history = consumer_history[consumer_history['gmv'] > 0]

In [ ]:
# reference_date (기준 날짜)
reference_date = pd.to_datetime("2016-02-28")

# Recency 계산
consumer_history['Recency'] = (reference_date - consumer_history['last_order']).dt.days

# 각 고객의 구매 빈도 계산 (주문 횟수)
frequency = consumer_history.groupby('cust_id')['order_date'].count().reset_index()
frequency.columns = ['cust_id', 'Frequency']

# 고객별 총 구매 금액 계산 
monetary = consumer_history.groupby('cust_id')['gmv'].sum().reset_index()
monetary.columns = ['cust_id', 'Monetary']

# Recency 데이터를 DataFrame으로 변환
recency = consumer_history[['cust_id', 'Recency']].drop_duplicates()

In [ ]:
# Recency, Frequency, Monetary 데이터를 합침
rfm_df = recency.merge(frequency, on='cust_id', how='left')
rfm_df = rfm_df.merge(monetary, on='cust_id', how='left')

# rfm_df에서 중복된 행 제거
rfm_df = rfm_df.drop_duplicates(subset=['cust_id'])

# RFM 데이터 확인
print(rfm_df.head())
print(rfm_df.info())

In [ ]:
# Group 1: 첫 구매일이 2015-09 ~ 2015-11인 고객
group1 = consumer_history[(consumer_history['First_purchase'] >= '2015-09-01') &
                          (consumer_history['First_purchase'] <= '2015-11-30')]

# Group 2: 첫 구매일이 2015-12 ~ 2016-02인 고객
group2 = consumer_history[(consumer_history['First_purchase'] >= '2015-12-01') &
                          (consumer_history['First_purchase'] <= '2016-02-28')]

# Group 1의 RFM 데이터 추출
group1 = group1['cust_id'].unique()
rfm_group1 = rfm_df[rfm_df['cust_id'].isin(group1)]

# Group 2의 RFM 데이터 추출
group2 = group2['cust_id'].unique()
rfm_group2 = rfm_df[rfm_df['cust_id'].isin(group2)]


In [ ]:
# 소수점 2자리까지 출력
pd.options.display.float_format = '{:,.2f}'.format

# Group 1 RFM 통계
print("Group 1 RFM Summary:")
print(rfm_group1.describe())

# Group 2 RFM 통계
print("\nGroup 2 RFM Summary:")
print(rfm_group2.describe())

## [마지막 일자 = 2016-02-28] 그룹 1, 2의 count가 일치하여 두 그룹 간의 비교가 보다 신뢰성 있게 이루어질 수 있다.

### Group 1: 구매 빈도와 마지막 거래일이 상대적으로 더 오래된 경향을 보임, 재구매 비율이 낮은 그룹
- 마지막 거래일: 평균 131일전 / 최대 179일전
- 구매 빈도: 평균 1.4회 / 최대 54회 / 75%의 고객들이 '2회 구매'
- 구매 금액: 평균 3,369달러 / 최대 2,227,654달러

> 최근거래는 오래된 상태이지만 한 번 구매시 상대적으로 고가의 제품을 구매했거나 여러 제품을 한꺼번에 구입했을 가능성

### Group 2:구매 빈도는 낮지만 마지막 거래일이 더 최근에 집중, 재구매 행동이 이루어진 그룹
- 마지막 거래일: 평균 75일전 / 최대 120일전
- 구매 빈도: 평균 1.1회 / 최대 80회 / 75%의 고객들이 '1회 구매'
- 구매 금액: 평균 3,088달러 / 최대 880,797달러

> 비교적 최근 거래가 기준일에 가까운 점이 특징이며, 재구매를 시도하는 고객들이 존재한다.   

---

In [ ]:
rfm_df.info()

In [ ]:
# Recency 계산
consumer_history['Recency'] = (reference_date - consumer_history['last_order']).dt.days

# 각 고객의 구매 빈도 계산 (주문 횟수)
frequency = consumer_history.groupby('cust_id')['order_date'].count().reset_index()
frequency.columns = ['cust_id', 'Frequency']

# 고객별 총 구매 금액 계산 
monetary = consumer_history.groupby('cust_id')['gmv'].sum().reset_index()
monetary.columns = ['cust_id', 'Monetary']

# Recency 데이터를 DataFrame으로 변환
recency = consumer_history[['cust_id', 'Recency']].drop_duplicates()

# Recency, Frequency, Monetary 데이터를 합침
rfm_df = recency.merge(frequency, on='cust_id', how='left')
rfm_df = rfm_df.merge(monetary, on='cust_id', how='left')

# rfm_df에서 중복된 행 제거
rfm_df = rfm_df.drop_duplicates(subset=['cust_id'])

# RFM 데이터 확인
print(rfm_df.head())

In [ ]:
# Frequency 값에 중복 제거
rfm_df['Frequency'] = rfm_df['Frequency'].astype(int)

# Recency 점수화 (값이 낮을수록 높은 점수)
recency_bins = [0, 30, 60, 100, np.inf]  # 0~30일 거래는 가장 최근, 120일 이상은 오래된 거래
recency_labels = [4, 3, 2, 1]  #4: 최근 거래 - 1: 예전거래 

rfm_df['Recency_Score'] = pd.cut(rfm_df['Recency'], bins=recency_bins, labels=recency_labels, include_lowest=True)


# Frequency를 4개의 구간으로 나누기 (적절한 bins 설정)
# 중복된 구간을 처리하기 위해 duplicates='drop'을 추가
frequency_bins = [0, 20, 40, 80, np.inf]  
frequency_labels = ['1', '2', '3', '4'] #4: 빈도 많음 - 1: 빈도 적음

rfm_df['Frequency_Score'] = pd.cut(rfm_df['Frequency'], bins=frequency_bins, labels=frequency_labels, include_lowest=True) 

# Monetary 점수화 (값이 높을수록 높은 점수)
monetary_bins = [0, 5000, 10000, 50000, np.inf]
monetary_labels = ['1', '2', '3', '4'] #4: 고지출 - 1: 저지출

rfm_df['Monetary_Score'] = pd.cut(rfm_df['Monetary'], bins=monetary_bins, labels=monetary_labels, include_lowest=True) 

# RFM 점수 결합 [444]: 우수 - [111]: 이탈
rfm_df['RFM_Score'] = rfm_df['Recency_Score'].astype(str) + rfm_df['Frequency_Score'].astype(str) + rfm_df['Monetary_Score'].astype(str)

# RFM 데이터 확인
rfm_df.head()

In [ ]:
# Recency를 기준으로 Group 1과 Group 2로 나누기 (기준을 평균으로 설정-최근 거래와 예전 거래 구분)
recency = rfm_df['Recency'].mean()
group_1 = rfm_df[rfm_df['Recency'] > recency] # 최근 거래일이 평균 보다 오래전
group_2 = rfm_df[rfm_df['Recency'] <= recency] # 최근 거래일이 평균 보다 가까움

In [ ]:
# Group 1 RFM 점수 계산
group_1['Recency_Score'] = pd.cut(group_1['Recency'], bins=recency_bins, labels=recency_labels, include_lowest=True)
group_1['Frequency_Score'] = pd.cut(group_1['Frequency'], bins=frequency_bins, labels=frequency_labels, include_lowest=True)
group_1['Monetary_Score'] = pd.qcut(group_1['Monetary'], 4, labels=[1, 2, 3, 4])

# Group 1 RFM 점수 결합
group_1['RFM_Score'] = group_1['Recency_Score'].astype(str) + group_1['Frequency_Score'].astype(str) + group_1['Monetary_Score'].astype(str)


In [ ]:
# Group 2 RFM 점수 계산
group_2['Recency_Score'] = pd.cut(group_2['Recency'], bins=recency_bins, labels=recency_labels, include_lowest=True)
group_2['Frequency_Score'] = pd.cut(group_2['Frequency'], bins=frequency_bins, labels=frequency_labels, include_lowest=True)
group_2['Monetary_Score'] = pd.qcut(group_2['Monetary'], 4, labels=[1, 2, 3, 4])

# Group 2 RFM 점수 결합
group_2['RFM_Score'] = group_2['Recency_Score'].astype(str) + group_2['Frequency_Score'].astype(str) + group_2['Monetary_Score'].astype(str)

In [ ]:
# 결과 확인
print("Group 1 RFM Summary:")
print(group_1[['cust_id', 'Recency', 'Frequency', 'Monetary', 'Recency_Score', 'Frequency_Score', 'Monetary_Score', 'RFM_Score']].head())

print("Group 2 RFM Summary:")
print(group_2[['cust_id', 'Recency', 'Frequency', 'Monetary', 'Recency_Score', 'Frequency_Score', 'Monetary_Score', 'RFM_Score']].head())

특징:
* 두 그룹 모두 거래 빈도가 1회에 마무리 됨 (F 저조)
* 그룹 1은 최근 거래일이 오래전 이지만 구매 금액이 상당히 높음 (R 저조 / M 최고 수준) = 그룹 내에서
  >이탈 고객으로 분석 : 재참여 유도
* 그룹 2은 최근 거래일이 비교적 가깝고 구매 금액도 높음 (R 높음 / M 높음) = 그룹 내에서
  >활성 고객으로 분석 : 충성도 강화 및 리텐션 전략

---

## 클러스터링(군집화) : 데이터에서 숨겨진 패턴이나 구조를 발견하고, 각 데이터 포인트를 적절한 군집에 할당하는 과정으로  거리 기반 알고리즘에서는 서로 다른 단위나 범위를 가진 변수들이 같은 비중으로 비교될 수 있도록 맞춰주는 것이 필요하다

정규 분포 확인 :

정규 분포 여부는 데이터가 평균을 중심으로
대칭적인 분포를 가지는지를 확인함으로써 평가할 수 있

* Q-Q Plot - 데이터가 대각선에 가까운 직선 형태를 띠면 정규 분포를 따를 가능성이 높다.다.

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats

stats.probplot(rfm_df['Recency'], dist="norm", plot=plt)
plt.show()

stats.probplot(rfm_df['Frequency'], dist="norm", plot=plt)
plt.show()

stats.probplot(rfm_df['Monetary'], dist="norm", plot=plt)
plt.show()

In [ ]:
# StandardScaler 변수들이 평균이 0, 표준편차가 1이 되도록 변환 / 데이터가 정규 분포를 따르는 경우에 적합
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
rfm_standardized = scaler.fit_transform(rfm_df[['Recency', 'Frequency', 'Monetary']])

In [ ]:
print(group_1.info())
print(group_2.info())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Recency vs Frequency
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Recency', y='Frequency', data=rfm_df, palette='Set1')
plt.title('Recency vs Frequency')
plt.xlabel('Recency')
plt.ylabel('Frequency')
plt.show()

# Recency vs Monetary
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Recency', y='Monetary', data=rfm_df, palette='Set1')
plt.title('Recency vs Monetary')
plt.xlabel('Recency')
plt.ylabel('Monetary')
plt.show()

# Frequency vs Monetary
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Frequency', y='Monetary', data=rfm_df, palette='Set1')
plt.title('Frequency vs Monetary')
plt.xlabel('Frequency')
plt.ylabel('Monetary')
plt.show()

K-means: 군집 중심(centroid)을 최적화하는 방식
- 군집이 구형이거나 규모가 균등할 때 효과적입니다.
- 군집 수(K)를 미리 정할 수 있을 때 사용합니다.

DBSCAN: 밀도 기반 군집화 알고리즘으로, 데이터의 밀도가 높은 지역을 군집으로 인식하고, 밀도가 낮은 지역을 노이즈로 간주합니다.
- 군집의 형태가 불규칙하고, 이상치가 있을 경우 유용합니다.
- 군집 수를 미리 지정할 필요가 없으며, 밀도 기반으로 클러스터를 찾습니다.

In [ ]:
from sklearn.cluster import KMeans

# K-means 클러스터링 적용
kmeans = KMeans(n_clusters=5, random_state=42)
rfm_df['Cluster'] = kmeans.fit_predict(rfm_standardized)

In [ ]:
group_1.info()

In [ ]:
# 그룹 1 데이터와 그룹 2 데이터를 따로 클러스터링 진행
group1_rfm = group_1[['Recency', 'Frequency', 'Monetary']]  # 그룹 1의 RFM 데이터
group2_rfm = group_2[['Recency', 'Frequency', 'Monetary']]  # 그룹 2의 RFM 데이터

In [ ]:
# 데이터 표준화 (StandardScaler)
scaler = StandardScaler()
group1_standardized = scaler.fit_transform(group1_rfm)
group2_standardized = scaler.fit_transform(group2_rfm)

In [ ]:
# 엘보우 방법을 사용하여 군집 수(K) 결정 (그룹 1)
inertia_values_group1 = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(group1_standardized)
    inertia_values_group1.append(kmeans.inertia_)

# 엘보우 그래프 그리기 (그룹 1)
plt.plot(k_range, inertia_values_group1, marker='o')
plt.title('Elbow Method - Group 1')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# 엘보우 방법을 사용하여 군집 수(K) 결정 (그룹 2)
inertia_values_group2 = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(group2_standardized)
    inertia_values_group2.append(kmeans.inertia_)

# 엘보우 그래프 그리기 (그룹 2)
plt.plot(k_range, inertia_values_group2, marker='o')
plt.title('Elbow Method - Group 2')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# 최적 군집 수를 찾은 후 (예: K=4) 각 그룹에 대해 K-means 클러스터링 진행
kmeans_group1 = KMeans(n_clusters=4, random_state=42)
group_1['Cluster'] = kmeans_group1.fit_predict(group1_standardized)

kmeans_group2 = KMeans(n_clusters=4, random_state=42)
group_2['Cluster'] = kmeans_group2.fit_predict(group2_standardized)

In [ ]:
# 각 그룹의 클러스터링 결과 확인 (예시로 Recency vs Monetary 시각화)
plt.figure(figsize=(8, 6))
for cluster in sorted(group_1['Cluster'].unique()):
    cluster_data = group_1[group_1['Cluster'] == cluster]
    plt.scatter(cluster_data['Recency'], cluster_data['Monetary'], label=f'Cluster {cluster}')
plt.title('Cluster Plot - Group 1')
plt.xlabel('Recency')
plt.ylabel('Monetary')
plt.legend(title='Cluster')
plt.colorbar()
plt.show()

plt.figure(figsize=(8, 6))
for cluster in sorted(group_2['Cluster'].unique()):
    cluster_data = group_2[group_2['Cluster'] == cluster]
    plt.scatter(cluster_data['Recency'], cluster_data['Monetary'], label=f'Cluster {cluster}')
plt.title('Cluster Plot - Group 2')
plt.xlabel('Recency')
plt.ylabel('Monetary')
plt.legend(title='Cluster')
plt.colorbar()
plt.show()

In [ ]:
# Group 1 클러스터별 RFM 평균 계산
group1_cluster_summary = group_1.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).rename(columns={'Recency': 'Avg_Recency', 'Frequency': 'Avg_Frequency', 'Monetary': 'Avg_Monetary'})

print("Group 1 Cluster Summary:")
print(group1_cluster_summary)

#### Group 1: 구매 빈도와 마지막 거래일이 상대적으로 더 오래된 경향을 보임, 재구매 비율이 낮은 그룹 

[평균값]- 클러스터 0: (127일 전)(횟수 1.4회)(32,294 달러) || **최근 거래는 오래되었지만, 구매금액이 매우 높은 고객**
- 클러스터 1: (134일 전)(횟수 4회)(6,053 달러) || **최근 거래는 오래되었지만, 비교적 높은 거래 빈도를 가진 고객**
- 클러스터 2: (121일 전)(횟수 1.1회)(1,486 달러) || **거래가 매우 드물고 구매 금액이 낮은 고객**
- 클러스터 3: (164일 전)(횟수 2회)(2,738달러) || **거래일이 매우 오래되고 구매 빈도는 낮지만 중간 정도의 구매 금액을 가진 고객**

[RFM 특징]
* 그룹 1은 최근 거래일이 오래전 이지만 구매 금액이 상당히 높음 (R 저조 / M 최고 수준) = 그룹 내에서
  >이탈 고객으로 분석 : 재참여 유도


In [ ]:
# Group 2 클러스터별 RFM 평균 계산
group2_cluster_summary = group_2.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).rename(columns={'Recency': 'Avg_Recency', 'Frequency': 'Avg_Frequency', 'Monetary': 'Avg_Monetary'})

print("Group 2 Cluster Summary:")
print(group2_cluster_summary)

#### Group 2: 구매 빈도는 낮지만 마지막 거래일이 더 최근에 집중, 재구매 행동이 이루어진 그룹

[평균값]
- 클러스터 0: (65일 전)(횟수 1.1회)(1,560 달러) || **최근 거래가 이루어졌지만, 구매 빈도와 금액은 다소 낮은 고객**
- 클러스터 1: (20일 전)(횟수 1.2회)(1,508 달러) || **매우 최근에 거래를 한 고객으로 구매 빈도와 금액이 중간 정도**
- 클러스터 2: (43일 전)(횟수 1.4회)(27,335 달러) || **중간 정도의 거래 빈도를 보이며, 상당히 높은 구매 금액을 가진 고객**
- 클러스터 3: (34일 전)(횟수 10회)(87,225달러) || **높은 구매 빈도와 구매 금액을 보이며, 최근 거래가 이루어진 VIP 고객**

[RFM 특징]
* 그룹 2은 최근 거래일이 비교적 가깝고 구매 금액도 높음 (R 높음 / M 높음) = 그룹 내에서
  >활성 고객으로 분석 : 충성도 강화 및 리텐션 전략

In [ ]:
# Group 1 클러스터별 RFM 분포
print("Group 1 RFM Distribution by Cluster:")
for cluster in group_1['Cluster'].unique():
    print(f"\nCluster {cluster} RFM Distribution:")
    print(group_1[group_1['Cluster'] == cluster][['Recency', 'Frequency', 'Monetary']].describe())

# Group 2 클러스터별 RFM 분포
print("Group 2 RFM Distribution by Cluster:")
for cluster in group_2['Cluster'].unique():
    print(f"\nCluster {cluster} RFM Distribution:")
    print(group_2[group_2['Cluster'] == cluster][['Recency', 'Frequency', 'Monetary']].describe())

[Group 1] = 이탈 그룹 : 활성화 그룹으로 전환

[Group 2] = 활성화 그룹 : 충성 고객 그룹으로 전환

## 목표: 매출 증대와 재구매율 상승
### 타켓 : Group 2 : 최근 거래와 재구매 행동이 이루어진 활성화 그룹을, 충성 고객 그룹으로 전환하여 매출 증대와 재구매율 상승

---

**Group 2 Cluster Summary:**

| Cluster | Avg_Recency | Avg_Frequency | Avg_Monetary |
|---------|-------------|---------------|--------------|
| 0       | 65.43       | 1.12          | 1,560.02     |
| 1       | 20.21       | 1.20          | 1,508.66     |
| 2       | 43.03       | 1.48          | 27,335.49    |
| 3       | 33.91       | 10.53         | 87,225.82    |


#### 1. 각 클러스터 별 구매 시기 (24시간 집계)

In [ ]:
consumer_history.info()

In [ ]:
group_2.info()

In [ ]:
# rfm_df에서 'cust_id'와 'Cluster' 컬럼이 제대로 존재하는지 확인
print(rfm_df.columns)  # 컬럼명 출력

# group_2에서 'Cluster' 컬럼이 제대로 존재하는지 확인
print(group_2.columns)  # 컬럼명 출력

In [ ]:
# group_2에서 'cust_id'를 이용해 unique한 고객 ID 목록을 추출
group_2_customers = group_2['cust_id'].unique()

# consumer_history에서 해당 고객들의 구매 이력 필터링
group_2_history = consumer_history[consumer_history['cust_id'].isin(group_2_customers)]

# 병합된 group_2 데이터프레임에서 'Cluster' 컬럼을 'Cluster_rfm'으로 변경하여 병합
group_2_history = group_2_history.merge(group_2[['cust_id', 'Cluster']], on='cust_id', how='left', suffixes=('', '_rfm'))

# 병합 후 데이터 확인
print(group_2_history.info())  # 병합된 데이터프레임의 정보 확인

In [ ]:
# 추가 분석을 위해 'Hour' 컬럼 생성 (구매 시각 추출)
group_2_history['Hour'] = group_2_history['order_date'].dt.hour  # 구매 시각을 'Hour' 컬럼으로 생성

# 클러스터별 시간대별 구매 건수 집계
hourly_purchases = group_2_history.groupby(['Cluster', 'Hour']).size().reset_index(name='Purchase_Count')


In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(data=hourly_purchases, x='Hour', y='Purchase_Count', hue='Cluster', marker='o', palette='Set1')
plt.title('Group 2 Clusters: Hourly Purchase Distribution')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Purchases')
plt.xticks(range(0, 24))
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

#### 1. 각 클러스터 별 구매 시기 (24시간 집계)
- 클러스터 0 / 클러스터 1: 비슷한 양상을 보이며, 낮 12시 이후부터 구매활동이 있으며, 21시에 높은 활동을 시도
  >광고나 프로모션을 21시경에 집중하는 전략
  
- 클러스터 2 / 클러스터 3: 비슷한 양상을 보이며, 특징적인 활동 시간 확인이 모호함
- >특정 시간대를 설정하는 것 보다 하루 종일 꾸준한 광고 노출이 효과적일 수 있다.

---

#### 2. 각 클러스터 별 선호 제품

In [ ]:
print(group_2_history['product_analytic_category'].unique())
print(group_2_history['product_analytic_sub_category'].unique()) # 해당 카테고리 선택
print(group_2_history['product_analytic_vertical'].unique())

In [ ]:
# 각 클러스터별로 제품 카테고리 별 구매 건수 집계
product_preferences = group_2_history.groupby(['Cluster', 'product_analytic_sub_category']).size().reset_index(name='Purchase_Count')

# 각 클러스터별로 상위 5개 선호 제품 카테고리 추출
top_products_per_cluster = product_preferences.groupby('Cluster').apply(lambda x: x.nlargest(5, 'Purchase_Count')).reset_index(drop=True)

print(top_products_per_cluster)

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=top_products_per_cluster, x='product_analytic_sub_category', y='Purchase_Count', hue='Cluster')
plt.title('Top 5 Preferred Products by Cluster')
plt.xlabel('Product Category')
plt.ylabel('Purchase Count')
plt.xticks(rotation=45)
plt.legend(title='Cluster')
plt.show()

#### 2. 각 클러스터 별 선호 제품 (product_analytic_sub_category)
- 클러스터 0 / 클러스터 1: 스피커 제품군 구매가 약 7만 건으로 가장 높고, 게임 액세서리와 카메라 액세서리는 약 3만 건, 홈 오디오는 약 1,7000건으로 확인되었다.
  > 스피커, 게임 액세서리, 카메라 액세서리, 오디오 제품에 대한 광고나 할인 및 가격 조정 정책을 시행
  
- 클러스터 2 / 클러스터 3: 특징적인 부분이 없고, 미약하나 클러스터 2에서 카메라 제품에 대한 구매가 약 18,000건으로 확인되었다.
  > 우수고객과 VIP 고객 그룹군으로 새로운 로얄티 제공 프로그램을 수립

---

#### 3. 각 클러스터 별 선호 가격대

In [ ]:
# 제품 가격 (product_mrp)을 이용하여 가격대 구간 나누기
price_bins = [0, 5000, 10000, 15000, 20000, 50000, 100000]  # 가격대 구간 (예시)
price_labels = ['0-5000', '5000-10000', '10000-15000', '15000-20000', '20000-50000', '50000-100000']
group_2_history['Price_Range'] = pd.cut(group_2_history['product_mrp'], bins=price_bins, labels=price_labels, right=False)

# 각 클러스터별 선호 가격대 집계
price_range_preferences = group_2_history.groupby(['Cluster', 'Price_Range']).size().reset_index(name='Purchase_Count')
print(price_range_preferences)


In [ ]:
# 각 클러스터별로 선호 가격대 시각화
plt.figure(figsize=(12, 8))
sns.barplot(data=price_range_preferences, x='Price_Range', y='Purchase_Count', hue='Cluster')
plt.title('Preferred Price Range by Cluster')
plt.xlabel('Price Range')
plt.ylabel('Purchase Count')
plt.xticks(rotation=45)
plt.legend(title='Cluster')
plt.show()

#### 2. 각 클러스터 별 선호 가격대
- 클러스터 0 / 클러스터 1: 0-5000달러 가격대에서 높은 구매 건수(저가 제품 선호), 고가 제품에 대한 관심은 낮음
  > 기간 한정 할인이나 묶음 할인을 제공하여 구매를 촉진
  
- 클러스터 2: 20000-50000달러 가격대에서 높은 구매 건수(고가 제품 선호)
  > VIP 프로그램을 통한 전용 할인이나 특별한 혜택 제공
  
- 클러스터 3
  

In [ ]:
# 가격 구간대
bins = [0, 5000, 10000, 20000, 50000]
labels = ['0-5000', '5000-10000', '10000-20000', '20000-50000']

# 'product_mrp' 컬럼을 기반으로 가격대 구분
consumer_history['Price_Range'] = pd.cut(consumer_history['product_mrp'], bins=bins, labels=labels, right=False)

# group_2와 consumer_history 병합
group_2_with_history = group_2.merge(consumer_history[['cust_id', 'product_analytic_sub_category', 'Price_Range']], on='cust_id', how='left')

# 각 클러스터 별로 선호 제품군과 가격대 분석
cluster_preference = group_2_with_history.groupby(['Cluster', 'product_analytic_sub_category', 'Price_Range']).size().reset_index(name='Purchase_Count')

# 결과 출력 (각 클러스터별 선호 제품군과 가격대별 구매 건수)
print(cluster_preference)

In [ ]:
# 시각화
plt.figure(figsize=(12, 8))
sns.barplot(data=cluster_preference, x='Price_Range', y='Purchase_Count', hue='product_analytic_sub_category', ci=None)
plt.title('Preferred Product Categories by Price Range for Each Cluster')
plt.xlabel('Price Range')
plt.ylabel('Purchase Count')
plt.xticks(rotation=45)
plt.legend(title='Product Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

#### 2. 각 클러스터 별 선호 제품 + 3. 각 클러스터 별 선호 가격대 대한 정보

- 클러스터 0 / 클러스터 1: 낮은 가격대에 존재하는 스피커, 게임 액세서리, 카메라 액세서리, 오디오 제품 선호도가 높음
- 클러스터 2: 높은 가격대에 존재하는 카메라 제품 선호도가 높음
- 클러스터 3: VIP 그룹은 현재까지 선호하는 제품 및 가격대 파악 모호

---

4. 각 클러스터 별 할인율에 따른 매출 / 재구매율 계산

In [ ]:
group_2_history.info()

In [ ]:
# 1. product_mrp만 적용했을 때 매출과 재구매율 계산
# 각 클러스터별로 product_mrp 기준으로 매출(gmv) 계산
cluster_sales_mrp = group_2_history.groupby('Cluster').agg(
    Total_Original_Sales=('gmv', 'sum'),  # 전체 매출
    Transaction_Count=('order_id', 'nunique')  # 고유 거래 수
).reset_index()

# 각 클러스터별 재구매율 계산 (재구매 고객 수 / 전체 고객 수)
repurchase_rate_by_cluster = []

# 각 클러스터별로 데이터를 분리하여 처리
for cluster_id in group_2_history['Cluster'].unique():
    cluster_data = group_2_history[group_2_history['Cluster'] == cluster_id]
    
    # 해당 클러스터 내에서 첫 번째 구매가 여러 번인 고객 찾기
    repeated_customers = cluster_data.groupby('cust_id').filter(lambda x: len(x) > 1)
    
    # 재구매 고객 수와 전체 고객 수 계산
    repeated_customers_count = repeated_customers['cust_id'].nunique()
    total_customers_count = cluster_data['cust_id'].nunique()
    
    # 재구매율 계산
    repurchase_rate = repeated_customers_count / total_customers_count
    
    # 결과 저장
    repurchase_rate_by_cluster.append({'Cluster': cluster_id, 'Repurchase_Rate': repurchase_rate})

# 결과를 DataFrame으로 변환
repurchase_rate_df = pd.DataFrame(repurchase_rate_by_cluster)

In [ ]:
# 클러스터별 매출과 재구매율을 합친 결과
cluster_sales_mrp = cluster_sales_mrp.merge(repurchase_rate_df, on='Cluster')

# 결과 출력
print("Cluster-wise Sales and Repurchase Rate (based on product_mrp):")
print(cluster_sales_mrp)

In [ ]:
# 2. 할인율 적용 시 매출과 재구매율 계산
# 할인율 계산: (product_mrp - gmv) / product_mrp * 100
group_2_history['discount_rate'] = ((group_2_history['product_mrp'] - group_2_history['gmv']) / group_2_history['product_mrp']) * 100

# 할인율이 -inf, inf인 값은 NaN으로 처리
group_2_history['discount_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)

# 할인율이 0보다 큰 값들(할인 적용된 데이터)만을 대상으로 매출과 거래 빈도 계산
discounted_sales = group_2_history[group_2_history['discount_rate'] > 0]
non_discounted_sales = group_2_history[group_2_history['discount_rate'] == 0]

# 할인율 적용 후 클러스터별 매출 계산
cluster_sales_discounted = discounted_sales.groupby('Cluster').agg(
    Discounted_Sales=('gmv', 'sum'),  # 할인된 매출
    Discounted_Transaction_Count=('order_id', 'nunique')  # 할인된 거래 수
).reset_index()

# 재구매율 계산 (할인 적용 후)
# 할인된 거래들에 대해서 재구매율을 계산
discounted_repeated_customers = discounted_sales.groupby('cust_id').filter(lambda x: len(x) > 1)
discounted_repeated_customers_count = discounted_repeated_customers['cust_id'].nunique()
discounted_repurchase_rate = discounted_repeated_customers_count / discounted_sales['cust_id'].nunique()

# 결과 출력
print("Cluster-wise Sales and Transaction Count with Discount:")
print(cluster_sales_discounted)
print("\nRepurchase Rate (with Discount):", discounted_repurchase_rate)

In [ ]:
from scipy import stats

# 3. 할인율이 매출과 재구매에 미치는 영향에 대한 t-검정

# 매출(gmv) 차이에 대한 t-검정 (할인 적용된 그룹 vs 비적용된 그룹)
t_stat_gmv, p_value_gmv = stats.ttest_ind(discounted_sales['gmv'], non_discounted_sales['gmv'], equal_var=False)

# 재구매율에 대한 t-검정: 재구매율 계산 (할인 적용된 그룹 vs 비적용된 그룹)
# 할인된 매출에서 재구매율을 계산하고, 비할인 그룹의 재구매율과 비교
# 할인 적용 여부에 따라 재구매 고객 수를 계산

# 할인된 그룹의 재구매율
discounted_repurchase_rate = discounted_repeated_customers['cust_id'].nunique() / discounted_sales['cust_id'].nunique()

# 비할인 그룹의 재구매율
non_discounted_repeated_customers = non_discounted_sales.groupby('cust_id').filter(lambda x: len(x) > 1)
non_discounted_repurchase_rate = non_discounted_repeated_customers['cust_id'].nunique() / non_discounted_sales['cust_id'].nunique()

# t-검정: 할인된 재구매율과 비할인 재구매율 차이 비교
t_stat_repurchase, p_value_repurchase = stats.ttest_ind(discounted_repurchase_rate, non_discounted_repurchase_rate)

# 결과 출력
print("\nT-test Result for GMV (Sales):")
print("t-statistic:", t_stat_gmv, ", p-value:", p_value_gmv)

print("\nT-test Result for Repurchase Rate:")
print("t-statistic:", t_stat_repurchase, ", p-value:", p_value_repurchase)

In [ ]:
# 각 클러스터별 미할인 매출과 할인 적용 매출
cluster_sales_data = {
    'Cluster': [0, 1, 2, 3],
    'Total_Sales_No_Discount': [264264996, 258497577, 560568914, 55126719],
    'Total_Sales_With_Discount': [251637001, 245879310, 537154723, 44731038]
}

# 데이터프레임으로 변환
import pandas as pd

cluster_sales_df = pd.DataFrame(cluster_sales_data)

# 매출 하락 비율 계산
cluster_sales_df['Sales_Decrease_Rate (%)'] = ((cluster_sales_df['Total_Sales_No_Discount'] - cluster_sales_df['Total_Sales_With_Discount']) / cluster_sales_df['Total_Sales_No_Discount']) * 100

# 결과 출력
print(cluster_sales_df[['Cluster', 'Sales_Decrease_Rate (%)']])

#### 4. 각 클러스터 별 할인율에 따른 매출 / 재구매율 계산

                 
|클러스터 | 총 매출(미할인) | 총 매출(할인 적용) | 매출 하락 비중(%) |
|---------|-------------|---------------|---------------|
| 0       | 264,264,996       | 251,637,001   | 4.78 |
| 1       | 258,497,577       | 245,879,310   | 4.88 |
| 2       | 560,568,914       | 537,154,723   | 4.18 | 
| 3       | 55,126,719        | 44,731,038    | 18.86|

- 할인이 매출에 미치는 영향으로는 p-value가 0.05보다 작은 값 (p-value = 0.0089)으로 나온 결과에서 할인이 매출에 유의미한 영향을 미친다고 결론을 내릴 수 있다.
- 코호트 분석에서 재구매율이 5.5% 미만으로 나타났음으로 할인과 재구매율 간에 유의미한 상관관계가 없을 가능성이 높다.

---

5. 각 클러스터 별 최적 배송시간

In [ ]:
group_2_history.head()

In [ ]:
# \N을 NaN으로 변환
group_2_history['deliverybdays'] = group_2_history['deliverybdays'].replace('\\N', np.nan)
group_2_history['deliverycdays'] = group_2_history['deliverycdays'].replace('\\N', np.nan)

# 변환 후 타입 확인
group_2_history.head(2)

In [ ]:
group_2_history.info()

In [ ]:
# 각 클러스터 별 최대 SLA 및 주문 수 계산
cluster_delivery_performance = group_2_history.groupby('Cluster').agg(
    max_sla=('sla', 'max'),  # 각 클러스터의 최대 SLA
    total_orders=('order_id', 'nunique')  # 각 클러스터의 고유 주문 수
).reset_index()

# 결과 확인
print(cluster_delivery_performance)

In [ ]:
# 'order_date'를 datetime 형식으로 변환
group_2_history['order_date'] = pd.to_datetime(group_2_history['order_date'])

#각 항목에 대해 최대 배송 소요 기간을 계산
group_2_history['max_sla_per_order'] = group_2_history.groupby('order_id')['sla'].transform('max')

# 'order_date'와 'sla'를 기준으로 예상 배송 완료일을 계산
# 최대 배송 소요 기간을 기반으로 "예상 배송일"을 계산
group_2_history['expected_delivery_date'] = group_2_history['order_date'] + pd.to_timedelta(group_2_history['max_sla_per_order'], unit='D')

group_2_history.head(5)

In [ ]:
# 각 클러스터별 평균 배송 소요 기간 계산
cluster_avg_sla = group_2_history.groupby('Cluster').agg(
    avg_sla=('max_sla_per_order', 'mean')  # 각 클러스터의 평균 최대 배송 소요 기간
).reset_index()

print(cluster_avg_sla)

In [ ]:
# 필요분석 - 실제 배송일이 존재한다면, 예상 배송일과 비교하여 지연 배송 여부를 파악


# 각 제품 카테고리별 평균 SLA 분석
product_delivery_performance = group_2_history.groupby('product_analytic_sub_category').agg(
    avg_sla=('max_sla_per_order', 'mean'),
    total_orders=('order_id', 'nunique')
).reset_index()

print(product_delivery_performance)

In [ ]:
# 각 클러스터의 선호 제품 카테고리와 평균 배송 소요 일수 결합
# 선호 제품 카테고리와 해당 카테고리의 평균 배송 소요 일수 결합
top_products_with_delivery = top_products_per_cluster.merge(
    product_delivery_performance[['product_analytic_sub_category', 'avg_sla']],
    left_on='product_analytic_sub_category', 
    right_on='product_analytic_sub_category', 
    how='left'
)

# 각 클러스터별 선호 제품 카테고리와 평균 배송 소요 일수 출력
print(top_products_with_delivery)

#### 5. 각 클러스터 별 최적 배송시간

전체 클러스터별 평균 배송일이 7일 이내로 확인됨, 필요분석 - 실제 배송일이 존재한다면, 예상 배송일과 비교하여 지연 배송 여부를 파악가능
 |Cluster|  max_sla|
 |-------|---------|
 |     0 |       35 |       
 |     1 |      175 |       
 |     2 |       22 |        
 |     3 |      145 |
 
 또한, 각 클러스터 별 최대 소요 일 수 확인 결과는 평균 배송 소요일수를 제외하고 특이케이스로 해당 값이 도출됨

In [ ]:
# 각 클러스터별 최대 SLA 값 설정
max_sla_values = {0: 35, 1: 175, 2: 22, 3: 145}

# 각 클러스터에서 최대 SLA를 가진 제품 필터링
long_delivery_products = group_2_history[
    group_2_history.apply(lambda row: row['sla'] == max_sla_values.get(row['Cluster'], None), axis=1)
]

# 결과 출력
print(long_delivery_products[['Cluster', 'product_analytic_sub_category', 'sla']])

In [ ]:
# 가격대 선호 데이터를 top_products_with_delivery에 결합
#top_products_with_delivery_and_price = top_products_with_delivery.merge(
#    price_range_preferences,
#    on='Cluster',
#    how='left'
#)

# 최종 결합된 데이터 출력
#print(top_products_with_delivery_and_price)

---

6. 각 클러스터 별 선호 광고

In [ ]:
ad_investment.head(12)

In [ ]:
group_2_history.info()

In [ ]:
# group_2_history 데이터의 연도와 월을 추출하여 기간을 확인
start_period = group_2_history['order_date'].min()
end_period = group_2_history['order_date'].max()

# ad_investment에서 group_2_history의 기간에 해당하는 연도와 월로 필터링
filtered_ad_investment = ad_investment[
    (ad_investment['Year'] == start_period.year) & (ad_investment['Month'] >= start_period.month) |
    (ad_investment['Year'] == end_period.year) & (ad_investment['Month'] <= end_period.month)
]

print("광고 투자 데이터 필터링 결과:")
filtered_ad_investment.head(7)

In [ ]:
# 월별 광고 투자액 계산
monthly_ad_investment = filtered_ad_investment.groupby(['Year', 'Month']).agg({
    'TV': 'sum', 'Digital': 'sum', 'Sponsorship': 'sum', 
    'Content Marketing': 'sum', 'Online marketing': 'sum', 
    ' Affiliates': 'sum', 'SEM': 'sum', 'Radio': 'sum', 'Other': 'sum'
}).reset_index()

# group_2_history에서 월별 매출(gmv) 계산
monthly_sales = group_2_history.groupby(['Year', 'Month'])['gmv'].sum().reset_index()

# 광고 투자와 매출을 합쳐서 분석
monthly_data = pd.merge(monthly_ad_investment, monthly_sales, on=['Year', 'Month'], how='left')

# 광고 투자와 매출의 상관 관계 분석
correlation = monthly_data.corr()

print("광고 투자와 매출의 상관 관계:")
print(correlation)